<a href="https://colab.research.google.com/github/RajuGuguloth/DL-Assignment-2/blob/main/DL_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setup and Imports**

In [3]:
# Import necessary libraries
import os
import random
import gc
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm.auto import tqdm
import wandb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix


# **Global Configs & Dataset Prep**

In [4]:
#  Device setup and image config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMG_SIZE = (224, 224)
CLASS_NAMES = ["Amphibia", "Animalia", "Arachnida", "Aves", "Fungi",
               "Insecta", "Mammalia", "Mollusca", "Plantae", "Reptilia"]

# Check if dataset is already available
if not os.path.exists("inaturalist_12K"):
    !wget -q https://storage.googleapis.com/wandb_datasets/nature_12K.zip -O nature_12K.zip
    !unzip -q nature_12K.zip
    !rm nature_12K.zip
    print("Dataset downloaded and extracted.")
else:
    print("Dataset already exists.")


✅ Dataset downloaded and extracted.


# ***DataLoader Preparation***

In [5]:
# Custom data loading with stratified validation
def prepare_dataloaders(train_path, val_path, batch_size):
    preprocessing = transforms.Compose([
        transforms.Resize(IMG_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
    ])

    # Load full training dataset
    full_train = datasets.ImageFolder(root=train_path, transform=preprocessing)
    class_indices = full_train.class_to_idx

    # Create balanced validation split
    val_indices, train_indices = [], []
    for class_label in class_indices.values():
        idx = [i for i, (_, y) in enumerate(full_train.samples) if y == class_label]
        train_idx, val_idx = train_test_split(idx, test_size=0.2, random_state=42)
        train_indices.extend(train_idx)
        val_indices.extend(val_idx)

    train_subset = Subset(full_train, train_indices)
    val_subset = Subset(full_train, val_indices)

    test_dataset = datasets.ImageFolder(root=val_path, transform=preprocessing)

    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

    return train_loader, val_loader, test_loader


# ***CNN Model (New Style)***

In [6]:
#  Modular CNN with flexible configuration
class CustomCNN(nn.Module):
    def __init__(self, conv_channels, kernel_sizes, dense_units, dropout_rate, activation, batch_norm):
        super().__init__()
        layers = []

        in_channels = 3
        for out_channels, k_size in zip(conv_channels, kernel_sizes):
            layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=k_size))
            layers.append(getattr(nn, activation)())
            if batch_norm:
                layers.append(nn.BatchNorm2d(out_channels))
            layers.append(nn.MaxPool2d(kernel_size=2))
            in_channels = out_channels

        self.features = nn.Sequential(*layers)

        dummy_input = torch.zeros(1, 3, *IMG_SIZE)
        dummy_out = self.features(dummy_input)
        flatten_dim = dummy_out.view(1, -1).shape[1]

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(flatten_dim, dense_units),
            nn.Dropout(dropout_rate),
            nn.Linear(dense_units, 10)
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)


In [16]:
#  Q1 & Q2: Calculate total computations and parameters manually

# Define your CNN architecture parameters
input_size = (64, 64, 3)   # height, width, channels (you can change this)
num_conv_layers = 5        # Number of conv layers
m = 32                     # Filters per conv layer
k = 3                      # Kernel size kxk
n = 256                    # Neurons in dense layer

def compute_total_computations_and_params(input_shape, num_conv_layers, m, k, n, output_classes=10):
    h, w, c = input_shape
    total_computations = 0
    total_params = 0

    # Convolution layers
    for i in range(num_conv_layers):
        output_h = h - k + 1
        output_w = w - k + 1
        # Each output element does k*k*c multiplications per filter
        macs_per_filter = output_h * output_w * (k * k * c)
        total_computations += macs_per_filter * m
        # Params: weights + bias per filter
        params_per_filter = (k * k * c) + 1
        total_params += params_per_filter * m

        # Update input for next layer
        h, w, c = output_h, output_w, m

    # Flatten the final feature map
    flatten_units = h * w * c

    # Dense layer
    total_computations += flatten_units * n
    total_params += flatten_units * n + n  # weights + bias

    # Output layer
    total_computations += n * output_classes
    total_params += n * output_classes + output_classes

    print("Total Multiply-Accumulate (MAC) operations:", f"{total_computations:,}")
    print("Total number of parameters (weights + biases):", f"{total_params:,}")

# Run it
compute_total_computations_and_params(
    input_shape=input_size,
    num_conv_layers=num_conv_layers,
    m=m,
    k=k,
    n=n,
    output_classes=10
)


Total Multiply-Accumulate (MAC) operations: 147,167,104
Total number of parameters (weights + biases): 23,928,586


# ***Accuracy & Evaluation Utilities***

In [17]:
#  Accuracy calculation
def compute_accuracy(model, dataloader):
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for imgs, labels in dataloader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            predictions = torch.argmax(outputs, dim=1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    return round(100 * correct / total, 2)


# **Training Function**

In [18]:
#  Model training loop with logging
def train_model(config=None):
    with wandb.init(config=config):
        config = wandb.config

        # Dataloaders
        train_loader, val_loader, test_loader = prepare_dataloaders(
            train_path="inaturalist_12K/train",
            val_path="inaturalist_12K/val",
            batch_size=config.batch_size
        )

        #  Model instantiation
        model = CustomCNN(
            conv_channels=config.conv_filters,
            kernel_sizes=config.kernel_sizes,
            dense_units=config.dense_units,
            dropout_rate=config.dropout,
            activation=config.activation,
            batch_norm=config.batch_norm
        ).to(device)

        #  Loss & Optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = getattr(optim, config.optimizer)(model.parameters(), lr=config.lr)

        # Training loop
        for epoch in range(config.epochs):
            model.train()
            running_loss = 0.0

            for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{config.epochs}", leave=False):
                images, labels = images.to(device), labels.to(device)

                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

            val_acc = compute_accuracy(model, val_loader)
            wandb.log({
                "epoch": epoch + 1,
                "loss": running_loss / len(train_loader),
                "val_accuracy": val_acc
            })

        # Final test accuracy
        test_acc = compute_accuracy(model, test_loader)
        wandb.log({"test_accuracy": test_acc})
        print(f" Final Test Accuracy: {test_acc}%")


# *Prediction Visualizer (Confusion Matrix + Images)*

In [19]:
#  Visualize predictions on test set
def visualize_predictions(model, dataloader, class_names):
    model.eval()
    images, labels, preds = [], [], []

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            predicted = torch.argmax(outputs, dim=1)

            images.extend(inputs.cpu())
            labels.extend(targets.cpu())
            preds.extend(predicted.cpu())

    #  Confusion matrix
    cm = confusion_matrix(labels, preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", xticklabels=class_names, yticklabels=class_names, cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("📊 Confusion Matrix")
    plt.show()

    # Show few sample predictions
    plt.figure(figsize=(12, 6))
    for i in range(6):
        img = images[i].permute(1, 2, 0) * 0.5 + 0.5  # Unnormalize
        true_label = class_names[labels[i]]
        pred_label = class_names[preds[i]]
        plt.subplot(2, 3, i + 1)
        plt.imshow(img)
        plt.title(f"True: {true_label}\nPred: {pred_label}")
        plt.axis("off")
    plt.tight_layout()
    plt.show()


# ***W&B Sweep Setup & Launch***

In [20]:
#  Define sweep config
sweep_config = {
    "method": "bayes",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "batch_size": {"values": [32, 64]},
        "epochs": {"value": 5},
        "lr": {"min": 1e-4, "max": 1e-2},
        "dropout": {"min": 0.2, "max": 0.3},
        "dense_units": {"values": [128, 256]},
        "conv_filters": {"values": [[16, 32, 64], [32, 64, 128]]},
        "kernel_sizes": {"values": [[3, 3, 3], [5, 3, 3]]},
        "optimizer": {"values": ["Adam", "SGD"]},
        "activation": {"values": ["ReLU", "LeakyReLU"]},
        "batch_norm": {"values": [True, False]},
    }
}

# Init sweep
sweep_id = wandb.sweep(sweep=sweep_config, project="DA6401_PartA")

# 🏃‍♂️ Start sweep agent (Uncomment to run)
# wandb.agent(sweep_id, function=train_model, count=5)


Create sweep with ID: tilapnfk
Sweep URL: https://wandb.ai/cs24m036-iit-madras-foundation/DA6401_PartA/sweeps/tilapnfk
